Setting up the dataframes and some useful variables

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import bisect
import glob
# Using 2016 data - stops and breakpoints
%matplotlib

import warnings
warnings.filterwarnings('ignore')

# Using the first quarter's data (beginning of Jan - end of March)
# list_of_dfs = []
# for i in range(12):
# #     df = pd.read_csv(str(i+1)+".csv")
#     # Use the following and comment the above for friday's dataset
#     df = pd.read_csv(str(i+1)+".csv")
#     list_of_dfs.append(df)

# final_df = pd.concat(list_of_dfs)

final_df = pd.read_csv("./travel_times_2016/fahrzeiten_soll_ist_20160103_20160109.csv")

# final_df.drop(['Unnamed: 0'], axis=1, inplace=True)
# Bus stop
df_haltepunkt = pd.read_csv("./travel_times_2016/haltepunkt.csv")

# Bus Station
df_haltestelle = pd.read_csv("./travel_times_2016/haltestelle.csv")

####################
# Some useful atts #
####################

# Target attributes - from: in seconds
target_arrival_from = "soll_an_von"

# Actual attributes - from: in seconds
actual_arrival_from = "ist_an_von"

# nominal departure from in seconds
target_departure_from = "soll_ab_von"

# actual  departure from in seconds
actual_departure_from = "ist_ab_von"
#######################################

#######################################
# Target attributes - from: in seconds
target_arrival_to = "soll_an_nach"

# Actual attributes - from: in seconds
actual_arrival_to = "ist_an_nach1"

# nominal departure from in seconds
target_departure_to = "soll_ab_nach"

# actual  departure from in seconds
actual_departure_to = "ist_ab_nach"
#####################################

#####################################

# key to haltestelle from and to
halt_id_from = "halt_id_von"
halt_id_to = "halt_id_nach"

############################
# key to haltepunkt from and to
halt_punkt_id_from = "halt_punkt_id_von"
halt_punkt_id_to = "halt_punkt_id_nach"

###########################

Using matplotlib backend: Qt5Agg


# Questions

1. Is there some sort of built in consideration for travel time of a given bus? It is crucial to find out, since it allows for stablishing a baseline for comparison

# Narrowing It Down


Going to narrow the results down to one bus for 1 day, and then generalize.

In [4]:
# variable to control whether results are narrowed or not
narrow = True

In [5]:
set(final_df['betriebsdatum'])

{'03.01.16',
 '04.01.16',
 '05.01.16',
 '06.01.16',
 '07.01.16',
 '08.01.16',
 '09.01.16'}

In [6]:
if narrow is True:# Narrow down to 1 day: 06.01.16 - DD.MM.YY
    date_to_narrow = "04.01.16"
    df_modified = final_df.loc[final_df['betriebsdatum'] == date_to_narrow]

In [7]:
# list of vehicle numbers
if narrow is True:
    print(set(df_modified['fahrzeug']))

{2048, 2050, 2051, 2052, 10245, 2054, 2055, 10247, 2057, 2058, 2059, 10246, 2061, 2062, 2063, 2064, 2065, 10254, 10255, 2067, 2069, 2070, 2071, 10264, 10263, 10265, 2075, 2076, 2074, 2078, 2079, 10271, 10273, 10274, 2081, 2084, 2083, 2085, 10276, 2087, 10279, 10282, 2090, 2091, 2092, 2093, 2094, 2088, 2096, 2097, 2098, 2099, 2101, 2102, 2103, 2104, 2105, 2107, 2108, 2109, 2110, 2111, 2112, 2113, 2114, 2116, 2117, 2118, 2119, 10312, 2120, 2121, 10310, 10311, 10313, 10331, 10333, 10334, 10335, 10336, 10337, 10338, 2066, 10403, 10404, 10405, 10406, 10407, 10408, 10409, 10410, 10411, 10412, 10413, 10414, 10415, 10416, 10417, 10418, 10419, 10420, 10421, 10422, 10423, 10424, 2072, 10426, 10427, 10428, 10429, 10430, 10431, 2077, 2080, 2082, 2086, 10510, 10511, 10512, 10513, 10514, 10515, 10517, 10518, 10519, 10521, 10523, 10524, 10525, 10526, 10528, 10531, 10532, 10533, 10535, 10536, 10537, 10539, 10540, 10541, 10542, 10543, 10544, 10545, 10546, 10547, 10548, 10549, 2095, 10552, 10553, 10554,

This cell shows the total number of recordings for a given vehicle, where the index is the vehicle number, let's pick the top index

In [8]:
df_modified.fahrzeug.value_counts()[:5]

10178    848
10180    846
10424    840
10164    840
10420    824
Name: fahrzeug, dtype: int64

In [9]:
# Narrow down to 1 vehicle: vehicle number
if narrow is True:
    vehicle_number = 10178
    df_modified = df_modified.loc[df_modified['fahrzeug'] == vehicle_number]

In [10]:
# Sorting by actual arrival time: to understand time structure
if narrow is True:
    final_df = df_modified.sort_values(by=[actual_arrival_from])
else:
    final_df = final_df.sort_values(by=[actual_arrival_from])

In [11]:
# Constructing the new df to save on computation

df_2 = final_df[[target_arrival_from, actual_arrival_from, target_departure_from, actual_departure_from,
             target_arrival_to, actual_arrival_to, target_departure_to, actual_departure_to, 'kurs']]


In [12]:
df_2.head()

,soll_an_von,ist_an_von,soll_ab_von,ist_ab_von,soll_an_nach,ist_an_nach1,soll_ab_nach,ist_ab_nach,kurs
196133,0,-17,12,-5,54,44,66,52,3
509028,54,44,66,52,114,97,132,101,3
56736,114,97,132,101,192,178,204,178,3
301352,192,178,204,178,282,259,300,276,3
305833,282,259,300,276,336,311,348,325,3


# Some definitions


# 1 - Time still
'time_still' is defined as departure at time n minus arrival at time n. Computes how long the bus is stationary. In the dataframe, it is denoted as 'target_stationary'

# 2 - Time travelling
'time_travelling' is defined as arrival at the next stop minus the departure at the current stop - in df denoted as 'target_travel_time'

In [13]:
target_stationary = 'target_stationary'
target_travel_time = 'target_travel_time'

df_2[target_stationary] = df_2[target_departure_from] - df_2[target_arrival_from]
df_2[target_travel_time] = df_2[target_arrival_to] - df_2[target_departure_from]

# Plotting

In [14]:
set(df_2['kurs'])

{3}

In [15]:
df_2 = df_2[df_2['kurs'] == 3]
df_2 = df_2.sort_values(by=[actual_arrival_from])
df_2.head()

,soll_an_von,ist_an_von,soll_ab_von,ist_ab_von,soll_an_nach,ist_an_nach1,soll_ab_nach,ist_ab_nach,kurs,target_stationary,target_travel_time
196133,0,-17,12,-5,54,44,66,52,3,12,42
509028,54,44,66,52,114,97,132,101,3,12,48
56736,114,97,132,101,192,178,204,178,3,18,60
301352,192,178,204,178,282,259,300,276,3,12,78
305833,282,259,300,276,336,311,348,325,3,18,36


In [16]:
# Plotting target arrival from  vs target_departure_from
%matplotlib
plt.title('target arrival from  vs target departure from')
plt.scatter(range(len(df_2)), df_2[target_arrival_from], c='k')
plt.scatter(range(len(df_2)), df_2[target_departure_from], c='y')
plt.xlabel('instance of recording')
plt.ylabel('time of the day')

Using matplotlib backend: Qt5Agg


Text(0, 0.5, 'time of the day')

# Another way to visualize

In [17]:
# Plotting target arrival from  vs target_departure_from

%matplotlib
plt.title('target arrival from  vs target departure from')
plt.scatter(range(0, 2 *len(df_2), 2), df_2[target_arrival_from], c='k')
plt.scatter(range(1, 2 *len(df_2), 2), df_2[target_departure_from], c='y')

Using matplotlib backend: Qt5Agg


# Filtering the course changes
Plotting the target arrival and departures of narrowed data - filtered the course changes - # cell below changes the df!

In [18]:
# Find the delay at each timestamp - just the actual - target
# side note: keep sequence change in mind!!!!
df_2['delay'] = df_2[actual_arrival_from] - df_2[target_arrival_from]

# Now, finding where the route is changing, i.e. the direction is changing or the sequence is resetting. These points
# seem to introduce a significant amount of delay, therefore going to remove these points, as it seems like the bus
# is waiting for a particular reason at the spot before changing its course

df_2[target_departure_from] = df_2[target_departure_from].shift(-1)

df_2['res3'] = df_2[target_departure_from] - df_2[target_departure_to]

df_2['course_change'] = df_2['res3'] > 0

# To find where the course is changing - later, these values get deleted for some plots
df_2 = df_2[df_2['course_change'] == False]

# Skip some of the larger delays - need to find out why these large (sometimes a few hours) happen
df_2 = df_2[abs(df_2['delay']) < 1800]

In [19]:
# Plotting target arrival from  vs target_departure_from

%matplotlib
plt.title('target arrival from  vs target departure from')
plt.scatter(range(0, 2 *len(df_2), 2), df_2[target_arrival_from], c='k')
# plt.hold(True)
plt.scatter(range(1, 2 *len(df_2), 2), df_2[target_departure_from], c='y')

Using matplotlib backend: Qt5Agg


In [20]:
# see how datapoints are distributed by kurs
df_2.kurs.value_counts()[:5]

3    819
Name: kurs, dtype: int64

In [21]:
# Box plot for delays

In [22]:
time_period = 1800
starting_time = 0
# indicates 30 mins intervals
# Number of bins:
n_bins = int(np.floor((86400 - starting_time)/time_period))
labels = [i for i in range(n_bins - 1)]
# Finding the correct bins:
list_of_bins = [starting_time + time_period * i for i in range(n_bins)]
df_2['intervals'] = pd.cut(df_2[target_arrival_from], bins=list_of_bins, retbins=False, labels=labels)

list_for_box_plot = []

for i in labels:
    temp = np.array(df_2[target_arrival_from][df_2['intervals'] == i])
    list_for_box_plot.append(temp)
plt.boxplot(list_for_box_plot)
plt.show()

In [23]:
# average stationary and travel time bar graphs

In [24]:
time_period = 1800
starting_time = 0

# indicates 30 mins intervals

# Number of bins:
n_bins = int(np.floor((86400 - starting_time)/time_period))
labels = [i for i in range(n_bins - 1)]

# Finding the correct bins:
list_of_bins = [starting_time + time_period * i for i in range(n_bins)]

df_2['intervals'] = pd.cut(df_2[target_arrival_from], bins=list_of_bins, retbins=False, labels=labels)

# to make the plotting more readible
labels_text = list_of_bins


avgs_stationary = []
# Averaging:
for i in labels:
    avgs_stationary.append(df_2[df_2['intervals'] == i].mean()[target_stationary])
for i in range(len(avgs_stationary)):
    if np.isnan(avgs_stationary[i]):
        avgs_stationary[i] = 0

avgs_travel_time = []
# Averaging:
for i in labels:
    avgs_travel_time.append(df_2[df_2['intervals'] == i].mean()[target_travel_time])
for i in range(len(avgs_travel_time)):
    if np.isnan(avgs_travel_time[i]):
        avgs_travel_time[i] = 0

# plotting stationary
f_1 = plt.figure(1)

plt.xlabel('time of the day')
plt.ylabel('stationary time in seconds - target')
plt.title('Target stationary times in seconds')
plt.xticks(labels, [
'0:00','0:30','','1:30','','2:30','','3:30','','4:30','','5:30','','6:30','','7:30','','8:30','',
'9:30','','10:30','','11:30','','12:30','','13:30','','14:30','',
'15:30','','16:30','','17:30','','18:30','','19:30','','20:30','',
'21:30','','22:30','','23:30'])
plt.bar(labels, avgs_stationary)
plt.show()

# plotting stationary
f_2 = plt.figure(2)

plt.xlabel('time of the day')
plt.ylabel('travel time in seconds - target')
plt.title('Target travel times in seconds')
plt.xticks(labels, [
'0:00','0:30','','1:30','','2:30','','3:30','','4:30','','5:30','','6:30','','7:30','','8:30','',
'9:30','','10:30','','11:30','','12:30','','13:30','','14:30','',
'15:30','','16:30','','17:30','','18:30','','19:30','','20:30','',
'21:30','','22:30','','23:30'])
plt.bar(labels, avgs_travel_time)
plt.show()

In [25]:
# Removing the mean:

for i in range(len(list_for_box_plot) - 1, 0, -1):
    if len(list_for_box_plot[i]) != 0:
        list_for_box_plot[i] = list_for_box_plot[i] - list_for_box_plot[i-1].mean()
    else:
        list_for_box_plot[i] = list_for_box_plot[i]


labels_text = [i for i in [
'0:00','0:30','','1:30','','2:30','','3:30','','4:30','','5:30','','6:30','','7:30','','8:30','',
'9:30','','10:30','','11:30','','12:30','','13:30','','14:30','',
'15:30','','16:30','','17:30','','18:30','','19:30','','20:30','',
'21:30','','22:30','','23:30']]
plt.boxplot(list_for_box_plot)

{'whiskers': [<matplotlib.lines.Line2D at 0x218d83e6a48>,
 'caps': [<matplotlib.lines.Line2D at 0x218d83e0548>,
 'boxes': [<matplotlib.lines.Line2D at 0x218d83e6608>,
 'medians': [<matplotlib.lines.Line2D at 0x218d83e2fc8>,
 'fliers': [<matplotlib.lines.Line2D at 0x218d83e21c8>,
 'means': []}

# Answer
Although kurs signifies the routes within the network, it is not an effective attribute to pivot the dataframe by to find the built in delays. We need a more granular attribute.

    Drop the course changes again and visualize by color - note, only available if not narrowed